In [ ]:
# Run in terminal or notebook
!pip install --upgrade langchain langchain-community faiss-cpu tiktoken openai pandas numpy chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninst

In [ ]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_KEY"
client = OpenAI(base_url="https://openai.vocareum.com/v1")

prompt = """
Generate 10 synthetic real estate listings.
Return ONLY valid CSV format (no backticks, no commentary).
Columns:
Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description

Rules:
- Bedrooms must be an integer between 1 and 5.
- Bathrooms must be an integer between 1 and 4.
- Price must be a realistic positive number.
- House Size must be a positive number in square feet.
- Add a proper in detail description for each estate.
- There should not be any None/Null/Void Cell in the CSV
"""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.5
)

csv_text = response.choices[0].message.content.strip()

with open("listings.csv", "w", encoding="utf-8") as f:
    f.write(csv_text)

print("CSV generated successfully → listings.csv")


CSV generated successfully → listings.csv


In [ ]:
import os
import csv
from openai import OpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader

# OpenAI client
client = OpenAI(base_url="https://openai.vocareum.com/v1")
# Load CSV into LangChain documents
loader = CSVLoader(file_path="listings.csv")
docs = loader.load()

'''
# Load CSV manually (CSVLoader is deprecated)
docs = []
with open("listings.csv", "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        text = "\n".join([f"{k}: {v}" for k, v in row.items()])
        docs.append(text)
'''
# Split text chunks
#splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=0)
#split_docs = splitter.create_documents(docs)

# Embeddings (new API)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_base="https://openai.vocareum.com/v1"
)

# Create ChromaDB
db = Chroma.from_documents(docs, embedding=embeddings)

print("ChromaDB created successfully.")


/tmp/ipython-input-2584762826.py:29: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


ChromaDB created successfully.


In [ ]:
query = "Find me a 3-bedroom house in a family-friendly area."

results = db.similarity_search(query, k=5)

for i, r in enumerate(results, 1):
    print(f"\n--- Result {i} ---")
    print(r.page_content)


--- Result 1 ---
Neighborhood: Suburbia
Price: 350000
Bedrooms: 4
Bathrooms: 3
House Size: 2500
Description: A lovely suburban home perfect for a growing family. This 4 bedroom
None: 3 bathroom house boasts a large kitchen,cozy fireplace,and a backyard patio for entertaining.

--- Result 2 ---
Neighborhood: Downtown
Price: 500000
Bedrooms: 3
Bathrooms: 2
House Size: 2000
Description: A charming townhouse located in the heart of downtown. This 3 bedroom
None: 2 bathroom home features a spacious living area and a beautiful backyard garden.

--- Result 3 ---
Neighborhood: Golf Course
Price: 400000
Bedrooms: 3
Bathrooms: 2
House Size: 2200
Description: Enjoy country club living in this 3 bedroom
None: 2 bathroom home overlooking the golf course. Features include a gourmet kitchen,outdoor patio,and access to community amenities.

--- Result 4 ---
Neighborhood: Ski Resort
Price: 700000
Bedrooms: 3
Bathrooms: 3
House Size: 2300
Description: Hit the slopes from this ski-in/ski-out chalet. Wit

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_base="https://openai.vocareum.com/v1"
)

# Build retriever
retriever = db.as_retriever(search_kwargs={"k": 5})

# Prompt template for RAG
prompt = ChatPromptTemplate.from_template("""
You are a helpful real estate assistant.

Use ONLY the property listings below to answer the user's question.

Listings:
{context}

User Question:
{query}

Answer:
""")

# RAG function
def rag_answer(query):
    # Retrieve docs
    docs = retriever.invoke(query)
    context = "\n\n".join([d.page_content for d in docs])

    # Create full prompt
    final_prompt = prompt.format(context=context, query=query)

    # Call LLM
    response = llm.invoke(final_prompt)
    return response.content

answer = rag_answer("Recommend a house for a family with children under 80 lakhs.")
print(answer)


I recommend the property in the Suburbia neighborhood. It is a lovely suburban home with 4 bedrooms, 3 bathrooms, and a house size of 2500 square feet, priced at 350000. Perfect for a growing family with children.


In [ ]:
from langchain_openai import ChatOpenAI

summary_llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_base="https://openai.vocareum.com/v1"
)

def summarize_preferences(questions, answers):
    text = ""
    for q, a in zip(questions, answers):
        text += f"Q: {q}\nA: {a}\n\n"

    prompt = f"""
Summarize the user's home preferences clearly.
Extract budget, bedrooms, neighborhood type, amenities, and any constraints.

Conversation:
{text}

Return summary in a detailed paragraph.
"""

    result = summary_llm.invoke(prompt)
    return result.content


In [ ]:
personal_questions = [
    "Ideal bedrooms?",
    "Preferred neighborhood?",
    "Maximum budget?",
    "Which amenities matter?",
    "Urban/suburban preference?"
]

answers = [
    "3 bedrooms",
    "Quiet, family-friendly area",
    "80 lakhs",
    "Parking and nearby schools",
    "Suburban"
]

summary = summarize_preferences(personal_questions, answers)
print(summary)


The user is looking for a home with 3 bedrooms in a quiet, family-friendly suburban neighborhood. Their maximum budget is 80 lakhs, and they prioritize amenities such as parking and nearby schools. They are not interested in an urban setting and prefer a more suburban environment for their ideal home.


In [ ]:
personalized_prompt = ChatPromptTemplate.from_template("""
You are a smart real estate advisor.

User Preferences:
{summary}

Listings:
{context}

User Question:
{query}

Answer in a friendly and helpful tone.
""")

def personalized_rag(query, summary):
    docs = retriever.invoke(query)
    context = "\n\n".join(d.page_content for d in docs)

    final_prompt = personalized_prompt.format(
        summary=summary,
        context=context,
        query=query
    )

    response = llm.invoke(final_prompt)
    return response.content


In [ ]:
final_answer = personalized_rag(
    "Recommend the best home for me.",
    summary
)

print(final_answer)

Based on your preferences for a quiet, family-friendly suburban neighborhood with 3 bedrooms and amenities like parking and nearby schools, I would recommend the home in the Golf Course neighborhood. This 3 bedroom, 2 bathroom home overlooks the golf course and offers a gourmet kitchen, outdoor patio, and access to community amenities. It fits within your budget of 80 lakhs and provides a peaceful suburban environment for you and your family. I believe this home would be a great fit for your needs and preferences. Let me know if you would like to schedule a viewing or have any more questions!
